In [12]:

# IMPORTS

import os
import time

import numpy as np
import cv2

import tensorflow as tf
from tensorflow.lite.python import interpreter


# Seed the Numpy
np.random.seed(100)

In [13]:
class TFLiteObjectDetector_BVA():
    
    def __init__(self, model_name="default_tflite_model_bva"):
        self.model_name = model_name



    def read_classes(self, class_file_path):
        """
        This function reads the coco.names file and create a list which has all the Class names
        """
        f = open(class_file_path)
        self.class_name_list = {}
        for i, val in enumerate(f):
            print("{} > {}".format(i,val))
            self.class_name_list.update({i: val.strip()})
            
        f.close()
   
        
        # FOR DEBUGGING
        print("\n[INFO] Number of Classes : {}".format(len(self.class_name_list)))

        # Creating a Uniform Distribution of Colors for Overlay of the Classes
        self.color_list = np.random.uniform(0, 255, size = (len(self.class_name_list),3))
        #FOR DEBUGGING
        print("[INFO] Number of Random Colors Generated : {}".format(len(self.color_list)))

    
    def load_model(self, model_path):
        """
        This functions loads the tflite model
        """
        self.model_path = os.path.abspath(model_path)

        print("\n[INFO] Loading the '{}' Model...".format(self.model_name))

        # Load the model and create the interpreter

        IMG_WIDTH,IMG_HEIGHT = 300, 300
        self.interpreter = tf.lite.Interpreter(model_path=self.model_path)
        self.interpreter.allocate_tensors()

        return None



    def configure_interpreter(self):
        """
        This function configures the interpreter 
        """
        # Get the Input and Output Details
        self.input_details = self.interpreter.get_input_details()
        print("input_details : ", self.input_details)
        print("\n\n")
        self.output_details = self.interpreter.get_output_details()
        print("output_details : ", self.output_details)
        print("\n\n")


        self.input_index = self.input_details[0]["index"]
        print("input_index : ", self.input_index)
        self.output_index = self.output_details[0]["index"]
        print("output_index : ", self.output_index)
        
        print("\n\n")
        self.input_tensor_shape = self.input_details[0]["shape"]
        self.IMG_HEIGHT =  self.input_tensor_shape[1]
        self.IMG_WIDTH = self.input_tensor_shape[2]
        print("IMG_HEIGHT : ", self.IMG_HEIGHT)
        print("IMG_WIDTH : ", self.IMG_WIDTH)

        return None


    
    def inference(self, input_image_path, min_conf_threshold=0.5):
        """ This function performance the inference on the image and returns the results.

        Args:
            input_image_path ([string]): Path to the Image for Inference
            min_conf_threshold (float, optional): Confidence Threshold of Prediction. Defaults to 0.5.

        Returns:
            [type]: [description]
            overlay_image (image) : image with the overlay
            FRAME_INFERENCE_DICT (dictionary) : dictionary of all detected classes and its associated info
            all_detected_objects (list) : list of all detected classes in the images
        """

        
        input_image_path = os.path.abspath(input_image_path)
        input_image_bgr = cv2.imread(input_image_path)
        overlay_image = input_image_bgr.copy()

        height, width, channel = input_image_bgr.shape 

        input_image_rgb = cv2.cvtColor(input_image_bgr,cv2.COLOR_BGR2RGB)
        input_image_rgb_resized = cv2.resize(input_image_rgb,(self.IMG_WIDTH,self.IMG_HEIGHT))
        
        #input_image_tensor = np.expand_dims(input_image_rgb_resized, axis=0).astype(np.float32)
        input_image_tensor = np.expand_dims(input_image_rgb_resized, axis=0).astype(np.uint8)
        print(input_image_tensor.shape)

        # Perform Inference
        self.interpreter.set_tensor(self.input_index, input_image_tensor)
        self.interpreter.invoke()

        # Retrieve Results
        #https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/TFLite_detection_image.py

        #Retrieve detection results
        boxes = self.interpreter.get_tensor(self.output_details[0]['index'])[0] # Bounding box coordinates of detected objects
        classes = self.interpreter.get_tensor(self.output_details[1]['index'])[0] # Class index of detected objects
        scores = self.interpreter.get_tensor(self.output_details[2]['index'])[0] # Confidence of detected objects
        num_det = self.interpreter.get_tensor(self.output_details[3]['index'])[0]

        # FOR DEBUG
        print ("boxes : ", boxes)
        print("classes : ", classes)
        print("scores : ", scores)
        print("num_detect : ", num_det)

        # DETECTION AND OVERLAY CODE
        # List to store all detections in an Image
        all_detected_objects = []
        all_object_instance_list = []

        # Dictionary of the entire frame
        FRAME_INFERENCE_DICT = {}
        all_object_dict = {}
        

                
        #Loop over all detections and draw detection box if confidence is above minimum threshold
        for i in range(len(scores)):
            if ((scores[i] > min_conf_threshold) and (scores[i] <= 1.0)):

                # Get bounding box coordinates and draw box
                # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
                
                ith_box = boxes[i]

                # Raw Values
                y_min, x_min, y_max, x_max = ith_box

            
                #bounding_box - raw 
                raw_bounding_box = {}
                raw_bounding_box['y_min'] = y_min 
                raw_bounding_box['x_min'] = x_min
                raw_bounding_box['y_max'] = y_max 
                raw_bounding_box['x_max'] = x_max
                
                # ORIGINAL CODE
                # ymin = int(max(1,(boxes[i][0] * imH)))
                # xmin = int(max(1,(boxes[i][1] * imW)))
                # ymax = int(min(imH,(boxes[i][2] * imH)))
                # xmax = int(min(imW,(boxes[i][3] * imW)))
                
                
                # Scaled Co - Ordinate for Overlay
                ymin = int(max(1,(y_min * height)))
                xmin = int(max(1,(x_min * width)))
                ymax = int(min(height,(y_max * height)))
                xmax = int(min(width,(x_max * width)))
                
                # scaled_bounding_box
                scaled_bounding_box = {}
                scaled_bounding_box['y_min'] = y_min 
                scaled_bounding_box['x_min'] = x_min
                scaled_bounding_box['y_max'] = y_max 
                scaled_bounding_box['x_max'] = x_max

                ith_class_index = int(classes[i])
                ith_class_name = self.class_name_list[ith_class_index] # Look up object name from "labels" array using class index
                ith_class_confidence = scores[i]
                ith_class_color = self.color_list[int(classes[i])]
                
                #overlay_text = '%s: %d%%' % (ith_class_name, int(scores[i]*100)) # Example: 'person: 72%'
                overlay_text = "{}: {}%".format(ith_class_name, ith_class_confidence)
                print(overlay_text)


                # FOR DEBUG
                print(i)
                print("ith_box : ", ith_box)
                print("ith_class_confidence :", ith_class_confidence)
                print("ith_class_index : ", ith_class_index)
                print("ith_class_name : ", ith_class_name)
                print("ith_class_color : ",ith_class_color)
                
                print("Overlay Text : {}".format(overlay_text))
        


                labelSize, baseLine = cv2.getTextSize(overlay_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
                label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
                
                cv2.rectangle(overlay_image, (xmin,ymin), (xmax,ymax), ith_class_color, 2)
                
                cv2.rectangle(overlay_image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
                cv2.putText(overlay_image, overlay_text, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

                
                # This list will accumalate all the detected classes
                all_detected_objects.append(ith_class_name)

                # This will be a new dictionary for each detection
                ith_detection_dict = {}
                ith_object_dict = {}
                ith_detection_dict['bounding_box'] = raw_bounding_box
                ith_detection_dict['scaled_bounding_box'] = scaled_bounding_box 
                ith_detection_dict['confidence'] = ith_class_confidence

                ith_object_dict[ith_class_name] = ith_detection_dict


                all_object_instance_list.append(ith_object_dict.copy())

            all_object_dict['height'] = height
            all_object_dict['width'] = width
            all_object_dict['channel'] = channel

            #all_object_dict['objects'] = ith_object_dict
            all_object_dict['objects'] = all_object_instance_list
            
            FRAME_INFERENCE_DICT = all_object_dict
            
            all_detected_objects = list(set(all_detected_objects))

        return overlay_image, FRAME_INFERENCE_DICT, all_detected_objects

In [1]:
TFLITE_CLASS_PATH = "../SSD of Vineet/coco.names"

TFLITE_MODEL_NAME = "tflite_dynamic_range_quantization"
TFLITE_MODEL_PATH = "ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/model_quan.tflite"
# TFLITE_MODEL_PATH = "data/pretrained_models/bva_tflite/dynamic_range_quantization_model.tflite"

In [3]:
TEST_FOLDER  = os.path.abspath('Car')
DUMMY_FILE_PATH  = os.path.abspath("Car/03099.jpeg")

In [5]:
def get_image_path_list(image_dir_path):

    all_file_paths = []

    acceptable_image_extensions = ('jpg', 'png', 'bmp', 'jpeg')


    test_folder_dir = os.path.abspath(image_dir_path)

    print("\n\n\n\n")
    for root, dirs, files in os.walk(test_folder_dir, topdown=True):  
        for file_name in files:
            # Skip if the files i .DS_Store. This is specific to Mac OS
            if file_name == ".DS_Store":
                continue
            if file_name.lower().endswith(acceptable_image_extensions):
                ith_file_path = os.path.abspath(os.path.join(root, file_name))
                # FOR DEBUG
                #print(ith_file_path)
                all_file_paths.append(ith_file_path)

    return all_file_paths

In [10]:
all_file_paths = get_image_path_list(TEST_FOLDER)
NUM_OF_IMAGES = len(all_file_paths)
print("NUM_OF_IMAGES : ", NUM_OF_IMAGES)






NUM_OF_IMAGES :  360


In [ ]:
model_load_start_time = time.time()
tflite_instance = TFLiteObjectDetector_BVA(TFLITE_MODEL_NAME)
print(tflite_instance.model_name)
tflite_instance.read_classes(TFLITE_CLASS_PATH)
tflite_instance.load_model(TFLITE_MODEL_PATH)
tflite_instance.configure_interpreter()
model_load_time = time.time() - model_load_start_time

In [20]:
# Perform Inference on the Dummy File to overcome Cold Start
overlay_image, FRAME_INFERENCE_DICT, all_detected_objects = tflite_instance.inference(DUMMY_FILE_PATH)
print(FRAME_INFERENCE_DICT)


# # TIME PROFILING

inference_start_time = time.time()

# #FOR DEBUG
# # for i in range(2):
# #     ith_image_path = all_file_paths[i]

for ith_image_path in all_file_paths:

    print("\n\n")
    print(ith_image_path)

    # Perform Inference on the ith_image_path
    # overlay_image, FRAME_INFERENCE_DICT, all_detected_objects = tflite_instance.inference(ith_image_path)
    overlay_image, FRAME_INFERENCE_DICT, all_detected_objects = tflite_instance.inference(ith_image_path,0.4)
    print(FRAME_INFERENCE_DICT)
    print(all_detected_objects)



inference_time = time.time() - inference_start_time 

FPS = (NUM_OF_IMAGES / inference_time)

print("\n\n\n")
print("------------------------------------------------------------------------------------")
print("                             MODEL TIME PROFILING ")
print("------------------------------------------------------------------------------------")
print()
print("MODEL LOAD TIME :", model_load_time)
print()
print("NUMBER OF IMAGES : ", NUM_OF_IMAGES)
print()
print("TOTAL INFERENCE TIME : ", inference_time)
print()
print("FPS : ", FPS)
print("------------------------------------------------------------------------------------")

AttributeError: 'TFLiteObjectDetector_BVA' object has no attribute 'IMG_WIDTH'